In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS #Lista predeterminada de palabras vacías en inglés

In [2]:
#data1=pd.read_csv("./deteccion_spam/datos/enronSpamSubset.csv")
#data2=pd.read_csv("./deteccion_spam/datos/lingSpam.csv")
data3=pd.read_csv("./deteccion_spam/datos/completeSpamAssassin.csv")

In [3]:
data=data3 #Haciendo pruebas por separado
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6046 entries, 0 to 6045
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6046 non-null   int64 
 1   Body        6045 non-null   object
 2   Label       6046 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 141.8+ KB


In [4]:
data.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


## Preprocesado

In [5]:
#data1.drop(["Unnamed: 0","Unnamed: 0.1"],inplace=True,axis=1)
#data2.drop("Unnamed: 0",inplace=True,axis=1)
data3.drop("Unnamed: 0",inplace=True,axis=1)

In [6]:
def preprocesamiento(data):
    data.drop_duplicates() #Eliminamos los correos duplicados (parece que no había)
    data.dropna() #Eliminamos los correos con valores nulos
    data = data.dropna(subset=['Body'])
    
    #División del conjunto de datos en entrenamiento y prueba
    X= data["Body"] #Atributos (sólo hay uno)
    y= data["Label"] #Etiquetas
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42) #Dividimos en conjunto de entrenamiento y de prueba (20% prueba)
    
    #Creamos una lista personalizada de palabras vacías
    stop_words_list = list(ENGLISH_STOP_WORDS)
    stop_words_list += ["subject"]

    #TOKENIZAMOS
    vectorizer=CountVectorizer(min_df=2, stop_words=stop_words_list)
    X_train=vectorizer.fit_transform(X_train)
    X_test=vectorizer.transform(X_test)
    #print(vectorizer.vocabulary_)
    
    return X_train, X_test, y_train, y_test, vectorizer

In [7]:
X_train, X_test, y_train, y_test, vectorizer= preprocesamiento(data)

In [8]:
X_train

<4836x26155 sparse matrix of type '<class 'numpy.int64'>'
	with 446500 stored elements in Compressed Sparse Row format>

In [17]:
import operator

# Obtener el vocabulario y sus recuentos de palabras
vocab = vectorizer.vocabulary_
word_counts = {word: X_train.getcol(idx).sum() for word, idx in vocab.items()}

# Ordenar el vocabulario por frecuencia de palabra (de mayor a menor)
sorted_vocab = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)

# Imprimir las palabras ordenadas por frecuencia
print("Palabras ordenadas por frecuencia:")
#for word, count in sorted_vocab:
    #print(word, count)

Palabras ordenadas por frecuencia:


# Naive Bayes

In [12]:
#Función donde entrenamos el clasificador Naive Bayes Multinomial
def clasificador_NBMultinomial(X_train, y_train):
    classifier=MultinomialNB()
    classifier.fit(X_train,y_train)
    
    return classifier

In [13]:
#Función que devuelve las medidas del clasificador
def medidas_NBMultinomial(X_test,y_test):
    accuracy=classifier.score(X_test,y_test)
    medidas=accuracy #añadir más
    return medidas

In [14]:
#Función para hacer nuevas predicciones con el modelo entrenado
def prediccion(new_emails, classifier, vectorizer):
    new_emails_transformed=vectorizer.transform(new_emails)
    predictions=classifier.predict(new_emails_transformed)
    return predictions

In [15]:
new_emails=[
    "Give me 5000$ or I will publish your private photos",
    "Pass by my office so we can discuss about your new assignment",
    "hello",
    "enron",
    "money"
    ]

In [16]:
classifier=clasificador_NBMultinomial(X_train, y_train)
medidas=medidas_NBMultinomial(X_test,y_test)
print(medidas)
nueva_prediccion=prediccion(new_emails, classifier, vectorizer)
print(nueva_prediccion)

0.9330024813895782
[1 0 1 0 1]
